### Flash Card Data To Firebase

In [1]:
#!pip install firebase_admin

In [2]:
#!pip install --upgrade pip

In [3]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [4]:
#!pip install pyrebase4

In [5]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [6]:
#import pyrebase
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import shutil

In [7]:
#import firebase_admin
#from firebase_admin import credentials

cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app = firebase_admin.initialize_app(cred)
#bucket = storage.bucket(app=app)

In [8]:
fire_db = firestore.client()

#### Read Data

In [9]:
data_root = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Root_Word_Translate_99.xlsx")
data_root

,sequence,root,root_arabic,root_arabic_frequency,surah,ayah,word rank,surah latin,arabic_simple,arabic,arabic_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,qwl,قول,1722,2,8,4,Al-Baqara,يقول,يَقُولُ,35,say,وہ کہتا ہے,वो कहता है,berkata,বলে,der,говорит:
1,1,qwl,قول,1722,2,11,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
2,1,qwl,قول,1722,2,11,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
3,1,qwl,قول,1722,2,13,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
4,1,qwl,قول,1722,2,13,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27372,99,nhr,نهر,113,85,11,11,Al-Burooj,الأنهار,الْأَنْهَارُ,40,the rivers,نہریں,नहरें,sungai-sungai,ঝর্ণাসমূহ,ırmaklar,"реки,"
27373,99,nhr,نهر,113,91,3,1,Ash-Shams,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
27374,99,nhr,نهر,113,92,2,1,Al-Lail,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
27375,99,nhr,نهر,113,93,10,4,Ad-Dhuha,تنهر,تَنْهَرْ,1,repel,جھڑکو,आप झिड़कये (उसे),kamu menghardik,তিরস্কার করো,azarlama,отгоняй!


In [10]:
data_word = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Word_Translate_300.xlsx")
data_word.reset_index(inplace=True)
data_word.rename(columns={"index":"sequence"}, inplace=True)
data_word["sequence"] = data_word["sequence"].apply(lambda x: x+1)
data_word

,sequence,arabic_simple,arabic_simple_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,من,2763,from,سے,जो,dari,থেকে,"-den,-dan",из
1,2,الله,2153,Allah,اللہ,अल्लाह,Allah,আল্লাহ,Allah,Аллах
2,3,في,1185,in,میں,ज़मीन में,dalam,মধ্যে,içinde,в
3,4,ما,1013,what,جو,जो,apa,যা,şeyleri,"то, что"
4,5,إن,966,Indeed,بیشک,बेशक,sesungguhnya,নিশ্চয়ই,şüphesiz,"Поистине,"
...,...,...,...,...,...,...,...,...,...,...
300,301,مريم,33,(of) Maryam,مریم,ऐ मरियम,Maryam,মারইয়ামের,Hz. Meryem,"Марьям,"
301,302,بعضهم,33,some of them,ان میں سے بعض,उनके बाज़ को,sebagian mereka,তাদের একে,bir kısmı,одни из них
302,303,ربكما,33,(of) your Lord,اپنے رب کی,अपने रब की,Tuhanmu berdua,তোমাদের উভয়ের রবের,Rabbinizin,Господа вашего (двоих)
303,304,وفي,33,and in,اور میں,और आख़िरत में,dan pada,এবং মধ্যে,ve içinde,и в


In [11]:
data_twogram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Twogram_Translate_Frequency.xlsx")
data_twogram.reset_index(inplace=True)
data_twogram.rename(columns={"index":"sequence"}, inplace=True)
data_twogram["sequence"] = data_twogram["sequence"].apply(lambda x: x+1)
data_twogram

,sequence,twogram_string,twogram_string_frequency,twogram_english,twogram_turkish,twogram_urdu,twogram_hindi,twogram_indonesian,twogram_bangla,twogram_russian
0,1,إن الله,205,God is,Allah,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
1,2,الذين آمنوا,184,who believed,kim inandı,جو ایمان لائے,जो विश्वास करता था,siapa yang percaya?,যারা বিশ্বাস করেছিল,кто верил
2,3,في الأرض,176,in Earth,Dünya'da,زمین میں,धरती में,di Bumi,পৃথিবীতে,на Земле
3,4,الذين كفروا,134,who disbelieved,kim inanmadı,جنہوں نے کفر کیا۔,जिसने अविश्वास किया,siapa yang tidak percaya?,যারা অবিশ্বাস করেছিল,кто не поверил
4,5,السماوات والأرض,133,Heavens and Earth,Gökler ve Dünya,آسمان اور زمین,स्वर्ग और पृथ्वी,Langit dan Bumi,আকাশ ও পৃথিবী,Небеса и Земля
...,...,...,...,...,...,...,...,...,...,...
195,196,من بعدهم,17,after them,onlardan sonra,ان کے بعد,उनके बाद,setelah mereka,তাদের পরে,после них
196,197,ولو شاء,17,if he wants,eğer isterse,اگر وہ چاہتا ہے,यदि वह चाहता है,jika dia mau,যদি সে চায়,если он хочет
197,198,إن كان,17,if,Eğer,اگر,अगर,jika,যদি,если
198,199,رسول الله,17,Messenger of God,Resulullah,خدا کے رسول,भगवान के दूत,Utusan Tuhan,রসূল সা,Посланник Бога


In [12]:
data_threegram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Threegram_Translate_Frequency.xlsx")
data_threegram.reset_index(inplace=True)
data_threegram.rename(columns={"index":"sequence"}, inplace=True)
data_threegram["sequence"] = data_threegram["sequence"].apply(lambda x: x+1)
data_threegram

,sequence,threegram_string,threegram_string_frequency,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,بسم الله الرحمن,114,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,2,يا أيها الذين,92,O you who,Ey sen kim,اے جو,हे तुम कौन,Wahai kamu siapa,হে আপনি যারা,"О ты, кто"
2,3,أيها الذين آمنوا,89,O you who believe,Ey inananlar,اے ایمان والو!,हे तुम जो विश्वास करते हो,Wahai orang-orang yang beriman,হে ঈমানদারগণ,"О вы, кто верит"
3,4,من دون الله,71,without God,Tanrı olmadan,خدا کے بغیر,भगवान के बिना,tanpa Tuhan,ঈশ্বর ছাড়া,без Бога
4,5,على كل شيء,52,for everything,herşey için,ہر چیز کے لئے,प्रत्येक वस्तु के लिए,untuk semuanya,সব কিছুর জন্য,За все
...,...,...,...,...,...,...,...,...,...,...
195,196,الأرض بعد موتها,8,earth after death,ölümden sonra dünya,موت کے بعد زمین,मृत्यु के बाद पृथ्वी,bumi setelah kematian,মৃত্যুর পরে পৃথিবী,земля после смерти
196,197,إن وعد الله,8,God's promise,Tanrı'nın vaadi,خدا کا وعدہ,भगवान का वादा,janji tuhan,ঈশ্বরের প্রতিশ্রুতি,обещание Бога
197,198,إن الله هو,8,God is,Tanrı,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
198,199,إلا أن يشاء,8,except that he wills,onun dilemesi dışında,سوائے اس کے کہ وہ چاہے,सिवाय इसके कि वह चाहता है,kecuali dia berkehendak,ব্যতীত যে সে ইচ্ছা করে,"кроме того, что он хочет"


In [13]:
data_ayah = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Plain_Ayahs_Frequency.xlsx")
data_ayah.reset_index(inplace=True)
data_ayah.rename(columns={"index":"sequence"}, inplace=True)
data_ayah["sequence"] = data_ayah["sequence"].apply(lambda x: x+1)
data_ayah

,sequence,arabic_sent_tanzil_plain,arabic_sent_frequency
0,1,فَبِأَيِّ آلَاءِ رَبِّكُمَا تُكَذِّبَانِ,31
1,2,وَيْلٌ يَوْمَئِذٍ لِلْمُكَذِّبِينَ,11
2,3,وَإِنَّ رَبَّكَ لَهُوَ الْعَزِيزُ الرَّحِيمُ,8
3,4,فَاتَّقُوا اللَّهَ وَأَطِيعُونِ,8
4,5,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ حم,7
...,...,...,...
6050,6051,وَكُلَّ إِنْسَانٍ أَلْزَمْنَاهُ طَائِرَهُ فِي ...,1
6051,6052,وَجَعَلْنَا اللَّيْلَ وَالنَّهَارَ آيَتَيْنِ ف...,1
6052,6053,وَيَدْعُ الْإِنْسَانُ بِالشَّرِّ دُعَاءَهُ بِا...,1
6053,6054,وَأَنَّ الَّذِينَ لَا يُؤْمِنُونَ بِالْآخِرَةِ...,1


#### Sending Data To Firestore

##### Root Data

In [18]:
data_root = data_root.iloc[0:15000,]  # data divide because of quota
data_root

,sequence,root,root_arabic,root_arabic_frequency,surah,ayah,word rank,surah latin,arabic_simple,arabic,arabic_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,qwl,قول,1722,2,8,4,Al-Baqara,يقول,يَقُولُ,35,say,وہ کہتا ہے,वो कहता है,berkata,বলে,der,говорит:
1,1,qwl,قول,1722,2,11,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
2,1,qwl,قول,1722,2,11,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
3,1,qwl,قول,1722,2,13,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
4,1,qwl,قول,1722,2,13,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,29,Hqq,حقق,287,2,42,6,Al-Baqara,الحق,الْحَقَّ,24,the truth,حق کو,हक़ को,benar,সত্যকে,hakkı,истину
14996,29,Hqq,حقق,287,2,61,55,Al-Baqara,الحق,الْحَقِّ,37,the truth,حق,हक़ के,kebenaran,সত্যের,gerçek,"истины,"
14997,29,Hqq,حقق,287,2,71,20,Al-Baqara,بالحق,بِالْحَقِّ,74,in truth,حق کے ساتھ,साथ हक़ के,dengan benar,যথাযথভাবে,hak ile,с истиной
14998,29,Hqq,حقق,287,2,91,17,Al-Baqara,الحق,الْحَقُّ,48,(is) the truth,حق ہے,हक़ है,benar,সত্য,gerçek,истина


In [15]:
# root
sequence_list = data_root["sequence"].to_list()
root_arabic_list = data_root["root_arabic"].to_list()
root_list = data_root["root"].to_list()
root_arabic_freq_list = data_root["root_arabic_frequency"].to_list()
# word 
arabic_simple_list = data_root["arabic_simple"].to_list()
arabic_list = data_root["arabic"].to_list()
trans_eng_list = data_root["translate_english"].to_list()
trans_urdu_list = data_root["translate_urdu"].to_list()
trans_hindi_list = data_root["translate_hindi"].to_list()
trans_indo_list = data_root["translate_indonesian"].to_list()
trans_bangla_list = data_root["translate_bangla"].to_list()
trans_tur_list = data_root["translate_turkish"].to_list()
trans_rus_list = data_root["translate_russian"].to_list()
surah_list = data_root["surah"].to_list()
ayah_list = data_root["ayah"].to_list()
word_rank_list = data_root["word rank"].to_list()

In [16]:
firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,surah_list,ayah_list,word_rank_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [17]:
# Sending Root Data
for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,surah,ayah,word_rank,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
    data = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            "surah":surah,
            "ayah":ayah,
            "word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Root_Data").document(f"root{sequence}").collection(f"{root}").document(f"{arabic_simple}").set(data)  # Not: Eger document ile bitiyorsa data set ile gönderiliyor.
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection("Detail").add(data)  # Not: Eğer collection ile bitiyorsa data add ile gönderiliyor ve collection dan sonraki
                                                                                         # document adları rastgele oluşturuluyor.                                                                                               

RetryError: Deadline of 60.0s exceeded while calling target function, last exception: 429 Quota exceeded.

In [ ]:
#fire_db.collection(u'cities').document(u'DC').delete()

##### Word Data

In [ ]:
data_word

In [ ]:
# word
sequence_list = data_word["sequence"].to_list()
arabic_simple_list = data_word["arabic_simple"].to_list()
arabic_simple_freq_list = data_word["arabic_simple_frequency"].to_list()
trans_eng_list = data_word["translate_english"].to_list()
trans_urdu_list = data_word["translate_urdu"].to_list()
trans_hindi_list = data_word["translate_hindi"].to_list()
trans_indo_list = data_word["translate_indonesian"].to_list()
trans_bangla_list = data_word["translate_bangla"].to_list()
trans_tur_list = data_word["translate_turkish"].to_list()
trans_rus_list = data_word["translate_russian"].to_list()

In [ ]:
firestore_sending_word_data = zip(sequence_list,arabic_simple_list,arabic_simple_freq_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [ ]:
# Sending Word Data
for sequence,arabic_simple,arabic_simple_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_word_data:
    data = {"arabic_simple":arabic_simple,
            "arabic_simple_freq":arabic_simple_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Word_Data").document(f"word{sequence}").collection(f"{arabic_simple}").document("detail").set(data)

##### Twogram Data

In [ ]:
data_twogram

In [ ]:
# twogram
sequence_list = data_twogram["sequence"].to_list()
twogram_string_list = data_twogram["twogram_string"].to_list()
twogram_string_freq_list = data_twogram["twogram_string_frequency"].to_list()
twogram_trans_eng_list = data_twogram["twogram_english"].to_list()
twogram_trans_urdu_list = data_twogram["twogram_urdu"].to_list()
twogram_trans_hindi_list = data_twogram["twogram_hindi"].to_list()
twogram_trans_indo_list = data_twogram["twogram_indonesian"].to_list()
twogram_trans_bangla_list = data_twogram["twogram_bangla"].to_list()
twogram_trans_tur_list = data_twogram["twogram_turkish"].to_list()
twogram_trans_rus_list = data_twogram["twogram_russian"].to_list()

In [ ]:
firestore_sending_twogram_data = zip(sequence_list,twogram_string_list,twogram_string_freq_list,twogram_trans_eng_list,twogram_trans_urdu_list,twogram_trans_hindi_list,twogram_trans_indo_list,twogram_trans_bangla_list,twogram_trans_tur_list,twogram_trans_rus_list)

In [ ]:
# Sending Twogram Data
for sequence,twogram,twogram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_twogram_data:
    data = {"twogram_simple":twogram,
            "twogram_freq":twogram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Twogram_Data").document(f"twogram{sequence}").collection(f"{twogram}").document("detail").set(data)

##### Threegram Data

In [ ]:
data_threegram

In [ ]:
# threegram
sequence_list = data_threegram["sequence"].to_list()
threegram_string_list = data_threegram["threegram_string"].to_list()
threegram_string_freq_list = data_threegram["threegram_string_frequency"].to_list()
threegram_trans_eng_list = data_threegram["threegram_english"].to_list()
threegram_trans_urdu_list = data_threegram["threegram_urdu"].to_list()
threegram_trans_hindi_list = data_threegram["threegram_hindi"].to_list()
threegram_trans_indo_list = data_threegram["threegram_indonesian"].to_list()
threegram_trans_bangla_list = data_threegram["threegram_bangla"].to_list()
threegram_trans_tur_list = data_threegram["threegram_turkish"].to_list()
threegram_trans_rus_list = data_threegram["threegram_russian"].to_list()

In [ ]:
firestore_sending_threegram_data = zip(sequence_list,threegram_string_list,threegram_string_freq_list,threegram_trans_eng_list,threegram_trans_urdu_list,threegram_trans_hindi_list,threegram_trans_indo_list,threegram_trans_bangla_list,threegram_trans_tur_list,threegram_trans_rus_list)

In [ ]:
# Sending Threegram Data
for sequence,threegram,threegram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_threegram_data:
    data = {"threegram_simple":threegram,
            "threegram_freq":threegram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}

    fire_db.collection("Threegram_Data").document(f"threegram{sequence}").collection(f"{threegram}").document("detail").set(data)

##### Ayah Data

#### Test

In [ ]:
data = {"name":"Ertugrul","surname":"Kurubal"}

In [ ]:
fire_db.collection("Root_Data").document("Person").collection("Detail").add(data)

In [ ]:
fire_db.collection("Root_Data2").document("Person2").set(data)

In [ ]:
# delete firestore
fire_db.collection(u"Root_Data").document(u"Person").delete()

In [ ]:
fire_db.collection(u"Root_Data").document(u"Person").update({})

#### Realtime Database Web App

In [ ]:
import pyrebase

In [ ]:
print(pyrebase.__version__)

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxx",
    "authDomain": "fxxxxx",
    "projectId": "fxxxx",
    "storageBucket": "flxxxx",
    "messagingSenderId": "xxxxxx",
    "appId": "1xxxx",
    "measurementId": "xxxxx"
    }  # Web App param from firebase

In [ ]:
firebase = pyrebase.initialize_app(firebaseConfig)

In [ ]:
db = firebase.database()

In [ ]:
data = {"name":"John", "age":25, "adress":["new york","los angeles"]}

In [ ]:
# db.push(data)  # create random name

In [ ]:
#db.child("Test_Data").child("Var").set(data)
db.child("Test_Data").set(data)

#### DataFrame Data To Firebase (Bucket, Storage)

##### 1 Way

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [ ]:
firebaseConfig = {
   "apiKey": "xxxxx",
   "authDomain": "xxxxx",
   "projectId": "xxxxx",
   "storageBucket": "xxxxx",
   "messagingSenderId": "xxxxx",
   "appId": "xxxxx",
   "databaseURL":"xxxxx"
}

In [ ]:
cred   = firebase_admin.credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app    = firebase_admin.initialize_app(cred, firebaseConfig)
bucket = storage.bucket(app=app)

In [ ]:
def upload_df(df, data_id):
    """
    Upload a Dataframe as a csv to Firebase Storage
    :return: storage_ref
    """

    # Storage location + extension
    storage_ref = data_id + ".csv"

    # Store locally
    df.to_csv(data_id)

    # Upload to Firebase Storage
    blob = bucket.blob(storage_ref)
    with open(data_id,'rb') as local_file:
        blob.upload_from_file(local_file)

    # Delete locally
    os.remove(data_id)

    return storage_ref

In [ ]:
df = pd.read_excel("")

##### 2 Way

In [ ]:
import pyrebase
import pandas

firebaseConfig = {
   "apiKey": "xxxxx",
   "authDomain": "xxxxx",
   "projectId": "xxxxx",
   "storageBucket": "xxxxx",
   "messagingSenderId": "xxxxx",
   "appId": "xxxxx",
   "databaseURL":"xxxxx"
}

firebase = pyrebase.initialize_app(firebaseConfig)

storage = firebase.storage()

df = pd.read_csv("/content/Future Prices.csv")

# here is the magic. Convert your csv file to bytes and then upload it
df_string = df.to_csv(index=False)
db_bytes = bytes(df_string, 'utf8')

fileName = "Future Prices.csv"

storage.child("predictions/" + fileName).put(db_bytes)